<a href="https://colab.research.google.com/github/bncolorado/Processing-ELTeC-corpus/blob/main/COLAB_notebooks/ELTeC_CodeSwitching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting code switching

One of the TEI tags used in ELTeC corpus is "foreign", that marks code switching. This notebook shows how to extract the languages other than Spanish (castilian) in ELTeC-SPA and to generates a word cloud.

## Loading ELTeC-SPA corpus in Colab

Only ELTeC-SPA will be loaded.

Corpus URL is https://github.com/COST-ELTeC/ELTeC-spa > "code" > copy "Download ZIP"

To load other collection (other languages): https://github.com/COST-ELTeC


In [ ]:
import zipfile

!wget "https://github.com/COST-ELTeC/ELTeC-spa/archive/refs/heads/master.zip" # paste here corpus url / Copiar aquíí la url del corpus

zip_ref = zipfile.ZipFile('master.zip', 'r') #Opens the zip file in "read" mode / abre el fichero comprimido en modo lectura
zip_ref.extractall() #Extracts files here (/content/) / Extrae los ficheros en la carpeta "content" (la actual)
zip_ref.close() #Cierro fichero
!rm master.zip #Removes ZIP to save space / Elimina el ZIP para ganar espacio

Now, the novels with the XML-TEI annotation are in this directory:

/content/ELTeC-spa-master/level1/

It is the level 1 that contains novels annotated with XML-TEI tags: TEI-Header, structure and other textual data. See encoding guidelines:

https://distantreading.github.io/Training/Budapest/encodingGuide-1.html

https://distantreading.github.io/Training/Budapest/encodingGuide-2.html#(1)

Level 0 will contain novels in plain texts (currently is empty)

Leve 2 will contain novels annotated with Part of Speech and lemmas (soon).

## Open each file and extract information about code switching

---



In [ ]:
import os
import re
from bs4 import BeautifulSoup

dir_in = "/content/ELTeC-spa-master/level1/" #This directory contains the novels / En este directorio están las novelas

foreign_lexicon = {} # Diccionario para almacenar las palabras y su idioma
print('Processing', dir_in)

for base, directorios, ficheros in os.walk(dir_in): #Recorre el directorio donde están las novelas
  for fichero in ficheros:
    ficheroEntrada = base + fichero
    directorio = base.split('/')[-1]
    if fichero[0:3] == "SPA": # Language ID / Identificador de idioma.
      with open(ficheroEntrada, 'r') as tei: #Opens the file / abre el fichero
        soup = BeautifulSoup(tei, 'xml') #Parse the XML / Procesa el XML
        #print("Processing", ficheroEntrada) #Only to see the process. Comment if it's not important.
        if soup.foreign != None:
          foreigns = soup.find_all('foreign') #Extract all tags "foreign" / Extra todas las etiquetas "foreign", que marcan el cambio de idioma en cada novela. 
          for item in foreigns: # Por cada etiqueta detectada, extrae el idioma y el fragmento de texto correspondiente. 
            #print(item)
            lang = item["xml:lang"] # Extract the name of the language / Extrae el nombre del idioma
            spam = item.text.lower() # Extract the words and tranform to lower case / Extrae las palabras y las pasa a minúsculas
            spam = re.sub('\n*','',spam) # A set of regular expressions to clean fragments. / Esto son expresiones regulares para limipiar el texto: elimina espacios sobrantes, cambios de línea, signos de puntuación, etc.
            spam = re.sub('\t*','',spam)
            spam = re.sub('  ','',spam)
            spam = re.sub('^ ','',spam)
            spam = re.sub(' $','',spam)
            spam = re.sub('\!','',spam)
            spam = re.sub('¡','',spam)
            spam = re.sub('\?','',spam)
            spam = re.sub('¿','',spam)
            spam = re.sub(',','',spam)
            spam = re.sub('\.*','',spam)
            spam = re.sub(';','',spam)
            spam = re.sub('«','',spam)
            spam = re.sub('»','',spam)
            spam = re.sub('\)','',spam)
            spam = re.sub('\(','',spam)
            spam = re.sub('^-','',spam)
            spam = re.sub('-$','',spam)
            spam = re.sub(':','',spam)
            
            if lang not in foreign_lexicon.keys(): # Create a dictionary: language and its words.
             foreign_lexicon[lang] = []            # Crea un diccionario. Idioma: fragmentos de texto en ese idioma.
             foreign_lexicon[lang].append(spam)    # Almacena la información en el diccionario
            else:
              foreign_lexicon[lang].append(spam)   # A partir de ahora, toda la información (idioma y sus textos) está en "foreign_lexicon"

for item in foreign_lexicon.items():
  print(item[0], item[1]) # Show langauges and words / Muestra el resultado: idioma más fragmentos de textos de ese idioma.


## Word list

In [ ]:
print("Generating frequent list")

NumberOfWords = {} # Diccionario para almacenar la cantidad de palabras de cada idioma

for item in foreign_lexicon.items(): # Bucle para extraer la información que hay en "foreign_lexicon"
  lang = item[0] # Guarda el nombre del idioma en la variable "lang"
  #print(lang)
  NumberOfWords[lang] = 0 # To count the number of words for each language / Para contar la cantidad de palabras. Comienza en 0 y luego lo irá incrementando cada vez que cuente una palabra nueva.
  
  words_out = '' # Variable tipo cadena para almacenar las palabras.
  for item in item[1]: # Bucle en los fragmento de textos extrídos de cada idioma.
    wrd = item.split(' ') # An extremely simple tokenization / Tokenizador simple para separar las palabas (por espacio en blanco)
    for w in wrd:
      #print(w)
      NumberOfWords[lang]+=1 #Add 1 to the number of words for this langauge / Suma 1 por cada nueva palabra
      words_out+=w+'\n' #Almacena la palabra en "words_out"

  out = open(lang+'_WordList.txt', 'w') #Opens a file in write mode ("w"). / Crea y abre un fichero nuevo (modo escritura) para guardar la lista de palabras de cada idioma.
  out.write(words_out) # Escribe las palabras en el fichero
  out.close() # Cierra el fichero
  # Esos ficheros están en el panel de la izquierda de COLAB, desde donde se pueden descargar.


print('Results. Number of words for each language:')
results = 'Number of words for each language:\n'
for item in NumberOfWords.items(): # Muestra los resultados finales.
  print(item[0], item[1])
  results+=item[0]+':\t'+str(item[1])+'\n'

out_results = open('results.txt', 'w') # Se genera un nuevo fichero para guardar y poder descargar los resultados finales.
out_results.write(results)
out_results.close()

### Plotting

In [ ]:
import matplotlib.pyplot as plt

# Toda la información está en "NumberOfWords", que se ha generado en el paso anterior.

x = NumberOfWords.keys() #Languages / Nombre de cada idioma
y = NumberOfWords.values() #Number of words for each lang. / Cantidad de palabras por cada idioma

plt.bar(x,y) #Creates the plot
plt.xlabel('Language')
plt.title('Number of word for each language')
plt.show()

## Show words as WordCloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator

#A custom stopwords list: / Lista de "stopword" creada "ad hoc". Se puede modificar (añadir o quitar palabras) o cargar otra.
stopwords = ['por', 'ch', 'ei', 'que','ei', 'io', 'se', 'de', 'est', 'ad', 'et', 'non', 'hoc', 'ex', 'le', 'la', 'qui', 'il', 'di', 'per', 'che', 'les', 'des', 'si', 'un']
for item in foreign_lexicon.items():
  text = ''
  for word in item[1]: # Recorre los fragmentos de texto y vuelve a separarlo en palabras.
    for w in word.split(' '):
      if w not in stopwords: # Fitro de "stopwords"
        text+=w+' ' #All words in a string / Si la palabra no está en la lista de "stopwords", la guarda en "text" como cadena de texto.

  wordcloud = WordCloud().generate(text) #Creates the wordcloud, one for each language  / Se genera la nube de palabras, una por cada idioma.

  # Display the generated image: / Muestra la nube generada.
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  print(item[0]) #Show the language
  plt.show() #Show the cloud

